# Get Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split    

In [2]:
data = pd.read_excel('BDADOS_Modelagem Futebol_IC2022.xlsx')

In [3]:
x2015_diff = data[data['Ano_Avaliação']==2015]

In [4]:
x2015_diff = data[['Status2020','GoldEscore_ModeloLinear','Ambidestro','CompetênciaPercebida', 'P_ConfiançaMotivação', 'P_Coping','P_ApoioFamilia', 'P_SaltoVertical', 'P_Agilidade', 'P_Velocidade20m','P_ResistênciaYOYO', 'P_Drible', 'P_Tática', 'IdadePVC']]

In [5]:
x2015_diff.head()

,Status2020,GoldEscore_ModeloLinear,Ambidestro,CompetênciaPercebida,P_ConfiançaMotivação,P_Coping,P_ApoioFamilia,P_SaltoVertical,P_Agilidade,P_Velocidade20m,P_ResistênciaYOYO,P_Drible,P_Tática,IdadePVC
0,0,50.989571,0,70,85.992891,88.493033,71.566115,74.857110,76.114793,22.965000,4.947147,41.293558,82.121362,15.836011
1,0,50.085655,0,80,34.826827,81.326706,57.925971,92.219616,83.891294,64.430875,24.825223,63.307174,90.490208,13.516524
2,0,44.474730,0,70,72.240468,90.490208,50.797831,44.828321,50.000000,51.595344,34.457826,50.000000,40.904588,14.320515
3,0,46.645113,0,80,8.534345,6.300836,18.141125,22.965000,61.791142,65.909703,9.012267,77.035000,40.904588,14.971890
4,0,45.482161,1,70,72.240468,35.569125,10.027257,88.876756,81.057035,98.808937,16.602325,44.038231,40.904588,12.867770


In [6]:
y = x2015_diff['Status2020']
y = pd.DataFrame([0 if y[i] == 1 else 1 for i in range(len(y))])

X = x2015_diff.drop(['Status2020'],axis=1)

In [7]:
data['Status2020'].value_counts() ## check the count of each class

0    132
1     33
Name: Status2020, dtype: int64

# Models

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import SVR
import xgboost as xgb
from tqdm import tqdm

In [9]:
model_list = {'SVM':SVC(),
              #'SVM_poy':SVC(kernel='poly',class_weight='balanced'),
                'RF': RandomForestClassifier(),
                'LR': LogisticRegression(),
                'GBC':GradientBoostingClassifier(),
                'MLP': MLPClassifier(),
                #'RF_1': RandomForestClassifier(n_estimators=1000,max_depth=3),
                'DT': DecisionTreeClassifier(),
                'XGB': xgb.XGBClassifier(),
                'KNN': KNeighborsClassifier()
              }

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
!pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 225 kB 5.3 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1


In [12]:
# creating the modeling dataset
from sklearn.datasets import make_classification

# data processing
  ## importation of pandas and numpy

# data visualization
  ## importation of matplotlib
import seaborn as sns

# model and performance
  ## importation of train_test_split
  ## importation of RandomForestClassifier
from sklearn.metrics import classification_report

# oversampling and undersampling
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from collections import Counter #count after processing the data

#predictions

In [13]:
import statistics
from sklearn.metrics import precision_recall_fscore_support
import pickle

In [14]:
columns = ['balancing_method','model','sampling_strategy','k-neighbors','m-neighbors','kind','accuracy','precision_0','precision_1',
               'recall_0','recall_1','f_0','f_1','support_0','support_1']

columnsUS = ['balancing_method','model','sampling_strategy','random_state','replacement','accuracy','precision_0','precision_1',
               'recall_0','recall_1','f_0','f_1','support_0','support_1']

#BASELINES

In [15]:
# Baselines
boot_samples = 30

results_matrix_resample = []

test_size = 0.2

for i in tqdm(range(boot_samples)):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) ##20% de validação
  for m_name, model in model_list.items():
    row = []
            
    bl_model = model.fit(X_train, y_train)
    y_pred = bl_model.predict(X_test)

    try:
      p,r,f,s = precision_recall_fscore_support(y_test, y_pred)

      row = ['None',m_name, 'none', 0, 0, 'none',
             accuracy_score(y_test,y_pred),p[0],p[1],r[0],r[1],f[0],f[1],s[0],s[1]]

    except:
      print(f'{m_name}\t{p}\t{r}\t{f}\t{s}')


    results_matrix_resample.append(row)

100%|██████████| 30/30 [00:16<00:00,  1.77it/s]


In [16]:
baseline_df = pd.DataFrame(results_matrix_resample)
baseline_df.columns = columns

In [17]:
baseline_df

,balancing_method,model,sampling_strategy,k-neighbors,m-neighbors,kind,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1
0,None,SVM,none,0,0,none,0.727273,0.000000,0.750000,0.000000,0.960000,0.000000,0.842105,8,25
1,None,RF,none,0,0,none,0.696970,0.333333,0.777778,0.250000,0.840000,0.285714,0.807692,8,25
2,None,LR,none,0,0,none,0.818182,0.666667,0.851852,0.500000,0.920000,0.571429,0.884615,8,25
3,None,GBC,none,0,0,none,0.696970,0.375000,0.800000,0.375000,0.800000,0.375000,0.800000,8,25
4,None,MLP,none,0,0,none,0.757576,0.000000,0.757576,0.000000,1.000000,0.000000,0.862069,8,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,None,GBC,none,0,0,none,0.787879,0.500000,0.827586,0.285714,0.923077,0.363636,0.872727,7,26
236,None,MLP,none,0,0,none,0.787879,0.500000,0.851852,0.428571,0.884615,0.461538,0.867925,7,26
237,None,DT,none,0,0,none,0.727273,0.375000,0.840000,0.428571,0.807692,0.400000,0.823529,7,26
238,None,XGB,none,0,0,none,0.787879,0.500000,0.851852,0.428571,0.884615,0.461538,0.867925,7,26


#OVERSAMPLING

#Borderline-SMOTE



In [19]:
boot_samples = 30

bs_results_matrix_resample = []

sampling_strategy_list=['not majority','all']
k_neighbors_list=[5,10,15]
m_neighbors_list=[5,10,15] 
kind_list=['borderline-1','borderline-2']
test_size = 0.2


# Bordeline SMOTE
for i in tqdm(range(boot_samples)):
  
  for sampling_strategy in sampling_strategy_list:
    for k_neighbors in k_neighbors_list:
      for m_neighbors in m_neighbors_list:
        for kind in kind_list:

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) ##20% de validação
        
          for m_name, model in model_list.items():
            row = []

            bs = BorderlineSMOTE(sampling_strategy=sampling_strategy, k_neighbors=k_neighbors, m_neighbors=m_neighbors, kind=kind)
            X_train_bs, y_train_bs = bs.fit_resample(X_train, y_train)

            bs_model = model.fit(X_train_bs, y_train_bs)
            y_pred = bs_model.predict(X_test)

            try:
              p,r,f,s = precision_recall_fscore_support(y_test, y_pred)


              row = ['BorderlineSMOTE',m_name, sampling_strategy, k_neighbors, m_neighbors, kind,
              accuracy_score(y_test,y_pred),p[0],p[1],r[0],r[1],f[0],f[1],s[0],s[1]]

            except:
              print(f'{m_name}\t{p}\t{r}\t{f}\t{s}')


            bs_results_matrix_resample.append(row)

100%|██████████| 30/30 [14:17<00:00, 28.59s/it]


In [20]:
SMOTE_df = pd.DataFrame(bs_results_matrix_resample)

In [21]:
SMOTE_df.columns = columns

In [22]:
SMOTE_df

,balancing_method,model,sampling_strategy,k-neighbors,m-neighbors,kind,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1
0,BorderlineSMOTE,SVM,not majority,5,5,borderline-1,0.636364,0.230769,0.900000,0.600000,0.642857,0.333333,0.750000,5,28
1,BorderlineSMOTE,RF,not majority,5,5,borderline-1,0.848485,0.500000,0.960000,0.800000,0.857143,0.615385,0.905660,5,28
2,BorderlineSMOTE,LR,not majority,5,5,borderline-1,0.818182,0.428571,0.923077,0.600000,0.857143,0.500000,0.888889,5,28
3,BorderlineSMOTE,GBC,not majority,5,5,borderline-1,0.848485,0.500000,0.925926,0.600000,0.892857,0.545455,0.909091,5,28
4,BorderlineSMOTE,MLP,not majority,5,5,borderline-1,0.727273,0.250000,0.880000,0.400000,0.785714,0.307692,0.830189,5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,BorderlineSMOTE,GBC,all,15,15,borderline-2,0.787879,0.500000,0.880000,0.571429,0.846154,0.533333,0.862745,7,26
8636,BorderlineSMOTE,MLP,all,15,15,borderline-2,0.727273,0.333333,0.814815,0.285714,0.846154,0.307692,0.830189,7,26
8637,BorderlineSMOTE,DT,all,15,15,borderline-2,0.727273,0.375000,0.840000,0.428571,0.807692,0.400000,0.823529,7,26
8638,BorderlineSMOTE,XGB,all,15,15,borderline-2,0.787879,0.500000,0.913043,0.714286,0.807692,0.588235,0.857143,7,26


#---

In [ ]:
dff_RF= dff.loc[dff['model']=='RF']

In [ ]:
print(dff_RF)

In [ ]:
results = dff
#results.columns = ['Model','Accuracy','Precision','Recall','BalancedAccuracy']
results.to_csv('dff_RF_Goldfit.csv')



#UNDERSAMPLING

#Random Undersampling

In [23]:
rus_results_matrix_resample = []

sampling_strategy_list=['majority','all']
random_state_list=[42]
replacement_list = [False, True]
test_size=0.2


# Random Undersampling
for i in tqdm(range(boot_samples)):
  
  for sampling_strategy in sampling_strategy_list:
    for random_state in random_state_list:
      for replacement in replacement_list:

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) ##20% de validação
        
          for m_name, model in model_list.items():
            row = []

            rus = RandomUnderSampler(sampling_strategy=sampling_strategy,random_state=random_state,replacement=replacement)
            X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

            rus_model = model.fit(X_train_rus, y_train_rus)
            y_pred = rus_model.predict(X_test)

            try:
              p,r,f,s = precision_recall_fscore_support(y_test, y_pred)


              row = ['Random Undersampling',m_name, sampling_strategy, random_state, replacement,
              accuracy_score(y_test,y_pred),p[0],p[1],r[0],r[1],f[0],f[1],s[0],s[1]]

            except:
              print(f'{m_name}\t{p}\t{r}\t{f}\t{s}')


            rus_results_matrix_resample.append(row)

100%|██████████| 30/30 [00:54<00:00,  1.82s/it]


In [24]:
print(rus_results_matrix_resample)

[['Random Undersampling', 'SVM', 'majority', 42, False, 0.696969696969697, 0.42857142857142855, 0.8947368421052632, 0.75, 0.68, 0.5454545454545454, 0.7727272727272727, 8, 25], ['Random Undersampling', 'RF', 'majority', 42, False, 0.696969696969697, 0.42857142857142855, 0.8947368421052632, 0.75, 0.68, 0.5454545454545454, 0.7727272727272727, 8, 25], ['Random Undersampling', 'LR', 'majority', 42, False, 0.7878787878787878, 0.5555555555555556, 0.875, 0.625, 0.84, 0.5882352941176471, 0.8571428571428572, 8, 25], ['Random Undersampling', 'GBC', 'majority', 42, False, 0.7575757575757576, 0.5, 0.9047619047619048, 0.75, 0.76, 0.6, 0.8260869565217391, 8, 25], ['Random Undersampling', 'MLP', 'majority', 42, False, 0.5151515151515151, 0.16666666666666666, 0.7142857142857143, 0.25, 0.6, 0.2, 0.6521739130434783, 8, 25], ['Random Undersampling', 'DT', 'majority', 42, False, 0.7272727272727273, 0.46153846153846156, 0.9, 0.75, 0.72, 0.5714285714285714, 0.7999999999999999, 8, 25], ['Random Undersampling'

In [25]:
RUS_df = pd.DataFrame(rus_results_matrix_resample)

In [26]:
RUS_df.columns = columnsUS

In [27]:
RUS_df

,balancing_method,model,sampling_strategy,random_state,replacement,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1
0,Random Undersampling,SVM,majority,42,False,0.696970,0.428571,0.894737,0.750,0.680000,0.545455,0.772727,8,25
1,Random Undersampling,RF,majority,42,False,0.696970,0.428571,0.894737,0.750,0.680000,0.545455,0.772727,8,25
2,Random Undersampling,LR,majority,42,False,0.787879,0.555556,0.875000,0.625,0.840000,0.588235,0.857143,8,25
3,Random Undersampling,GBC,majority,42,False,0.757576,0.500000,0.904762,0.750,0.760000,0.600000,0.826087,8,25
4,Random Undersampling,MLP,majority,42,False,0.515152,0.166667,0.714286,0.250,0.600000,0.200000,0.652174,8,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,Random Undersampling,GBC,all,42,True,0.757576,0.333333,0.916667,0.600,0.785714,0.428571,0.846154,5,28
956,Random Undersampling,MLP,all,42,True,0.484848,0.125000,0.823529,0.400,0.500000,0.190476,0.622222,5,28
957,Random Undersampling,DT,all,42,True,0.696970,0.222222,0.875000,0.400,0.750000,0.285714,0.807692,5,28
958,Random Undersampling,XGB,all,42,True,0.757576,0.285714,0.884615,0.400,0.821429,0.333333,0.851852,5,28


#Near Miss Undersampling

In [28]:
nm_results_matrix_resample = []

sampling_strategy_list=['majority','all'] #ver se usamos todos os parâmetros
version_list=[1,2,3]
n_neighbors_list=[3,5,10] ##investigar o melhor conjunto de vizinhos
n_neighbors_ver3_list=[3,5,10] 
test_size=0.2


# Random Undersampling
for i in tqdm(range(boot_samples)):
  
  for sampling_strategy in sampling_strategy_list:
    for version in version_list:
      for n_neighbors in n_neighbors_list:
        for n_neighbors_ver3 in n_neighbors_ver3_list:


          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) ##20% de validação
        
          for m_name, model in model_list.items():
            row = []

            nm = NearMiss(sampling_strategy=sampling_strategy,version=version,n_neighbors=n_neighbors,n_neighbors_ver3=n_neighbors_ver3)
            X_train_nm, y_train_nm = nm.fit_resample(X_train, y_train)

            nm_model = model.fit(X_train_nm, y_train_nm)
            y_pred = nm_model.predict(X_test)

            try:
              p,r,f,s = precision_recall_fscore_support(y_test, y_pred)


              row = ['NearMiss_Undersampling',m_name, sampling_strategy, random_state, replacement,
              accuracy_score(y_test,y_pred),p[0],p[1],r[0],r[1],f[0],f[1],s[0],s[1]]

            except:
              print(f'{m_name}\t{p}\t{r}\t{f}\t{s}')


            nm_results_matrix_resample.append(row)

100%|██████████| 30/30 [11:34<00:00, 23.16s/it]


In [29]:
#print(nm_results_matrix_resample)

In [30]:
NM_df = pd.DataFrame(nm_results_matrix_resample)

In [32]:
NM_df.columns = columnsUS

In [33]:
NM_df

,balancing_method,model,sampling_strategy,random_state,replacement,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1
0,NearMiss_Undersampling,SVM,majority,42,True,0.696970,0.230769,1.000000,1.000000,0.666667,0.375000,0.800000,3,30
1,NearMiss_Undersampling,RF,majority,42,True,0.727273,0.250000,1.000000,1.000000,0.700000,0.400000,0.823529,3,30
2,NearMiss_Undersampling,LR,majority,42,True,0.696970,0.181818,0.954545,0.666667,0.700000,0.285714,0.807692,3,30
3,NearMiss_Undersampling,GBC,majority,42,True,0.757576,0.142857,0.923077,0.333333,0.800000,0.200000,0.857143,3,30
4,NearMiss_Undersampling,MLP,majority,42,True,0.636364,0.153846,0.950000,0.666667,0.633333,0.250000,0.760000,3,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,NearMiss_Undersampling,GBC,all,42,True,0.666667,0.333333,0.944444,0.833333,0.629630,0.476190,0.755556,6,27
12956,NearMiss_Undersampling,MLP,all,42,True,0.454545,0.200000,0.846154,0.666667,0.407407,0.307692,0.550000,6,27
12957,NearMiss_Undersampling,DT,all,42,True,0.606061,0.266667,0.888889,0.666667,0.592593,0.380952,0.711111,6,27
12958,NearMiss_Undersampling,XGB,all,42,True,0.575758,0.250000,0.882353,0.666667,0.555556,0.363636,0.681818,6,27


In [34]:
m = pd.merge(baseline_df, SMOTE_df, how = 'outer')

In [35]:
m

,balancing_method,model,sampling_strategy,k-neighbors,m-neighbors,kind,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1
0,None,SVM,none,0,0,none,0.727273,0.000000,0.750000,0.000000,0.960000,0.000000,0.842105,8,25
1,None,RF,none,0,0,none,0.696970,0.333333,0.777778,0.250000,0.840000,0.285714,0.807692,8,25
2,None,LR,none,0,0,none,0.818182,0.666667,0.851852,0.500000,0.920000,0.571429,0.884615,8,25
3,None,LR,none,0,0,none,0.818182,0.666667,0.851852,0.500000,0.920000,0.571429,0.884615,8,25
4,None,GBC,none,0,0,none,0.696970,0.375000,0.800000,0.375000,0.800000,0.375000,0.800000,8,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8875,BorderlineSMOTE,LR,all,15,15,borderline-2,0.696970,0.384615,0.900000,0.714286,0.692308,0.500000,0.782609,7,26
8876,BorderlineSMOTE,GBC,all,15,15,borderline-2,0.787879,0.500000,0.880000,0.571429,0.846154,0.533333,0.862745,7,26
8877,BorderlineSMOTE,MLP,all,15,15,borderline-2,0.727273,0.333333,0.814815,0.285714,0.846154,0.307692,0.830189,7,26
8878,BorderlineSMOTE,XGB,all,15,15,borderline-2,0.787879,0.500000,0.913043,0.714286,0.807692,0.588235,0.857143,7,26


In [36]:
n = pd.merge(RUS_df, NM_df, how = 'outer')

In [37]:
n

,balancing_method,model,sampling_strategy,random_state,replacement,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1
0,Random Undersampling,SVM,majority,42,False,0.696970,0.428571,0.894737,0.750,0.68,0.545455,0.772727,8,25
1,Random Undersampling,RF,majority,42,False,0.696970,0.428571,0.894737,0.750,0.68,0.545455,0.772727,8,25
2,Random Undersampling,LR,majority,42,False,0.787879,0.555556,0.875000,0.625,0.84,0.588235,0.857143,8,25
3,Random Undersampling,GBC,majority,42,False,0.757576,0.500000,0.904762,0.750,0.76,0.600000,0.826087,8,25
4,Random Undersampling,MLP,majority,42,False,0.515152,0.166667,0.714286,0.250,0.60,0.200000,0.652174,8,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,NearMiss_Undersampling,LR,all,42,True,0.575758,0.333333,0.866667,0.750,0.52,0.461538,0.650000,8,25
13916,NearMiss_Undersampling,XGB,all,42,True,0.666667,0.411765,0.937500,0.875,0.60,0.560000,0.731707,8,25
13917,NearMiss_Undersampling,KNN,all,42,True,0.606061,0.352941,0.875000,0.750,0.56,0.480000,0.682927,8,25
13918,NearMiss_Undersampling,RF,all,42,True,0.727273,0.444444,0.833333,0.500,0.80,0.470588,0.816327,8,25


In [38]:
g = pd.merge(m, n, how = 'outer')

In [39]:
g

,balancing_method,model,sampling_strategy,k-neighbors,m-neighbors,kind,accuracy,precision_0,precision_1,recall_0,recall_1,f_0,f_1,support_0,support_1,random_state,replacement
0,None,SVM,none,0.0,0.0,none,0.727273,0.000000,0.750000,0.000,0.96,0.000000,0.842105,8,25,NaN,NaN
1,None,RF,none,0.0,0.0,none,0.696970,0.333333,0.777778,0.250,0.84,0.285714,0.807692,8,25,NaN,NaN
2,None,LR,none,0.0,0.0,none,0.818182,0.666667,0.851852,0.500,0.92,0.571429,0.884615,8,25,NaN,NaN
3,None,LR,none,0.0,0.0,none,0.818182,0.666667,0.851852,0.500,0.92,0.571429,0.884615,8,25,NaN,NaN
4,None,GBC,none,0.0,0.0,none,0.696970,0.375000,0.800000,0.375,0.80,0.375000,0.800000,8,25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,NearMiss_Undersampling,LR,all,NaN,NaN,NaN,0.575758,0.333333,0.866667,0.750,0.52,0.461538,0.650000,8,25,42.0,True
22796,NearMiss_Undersampling,XGB,all,NaN,NaN,NaN,0.666667,0.411765,0.937500,0.875,0.60,0.560000,0.731707,8,25,42.0,True
22797,NearMiss_Undersampling,KNN,all,NaN,NaN,NaN,0.606061,0.352941,0.875000,0.750,0.56,0.480000,0.682927,8,25,42.0,True
22798,NearMiss_Undersampling,RF,all,NaN,NaN,NaN,0.727273,0.444444,0.833333,0.500,0.80,0.470588,0.816327,8,25,42.0,True


In [44]:
g.to_csv('GoldFitSoccer_Balanceamento_2022_12_22.csv')

In [42]:
#print(g)

In [43]:
g.to_excel('GoldFitSoccer_Balanceamento_2022_12_22.xlsx')